# Семинар 10. Трансформер 🤖 🦾

- Был представлен в статье [Attention is all you need](https://papers.nips.cc/paper/7181-attention-is-all-you-need.pdf) в 2017 г.
- Seq2seq модель (состоит из энкодера и декодера)
- Позволяет добиться более высокого качества перевода и быстрее обучается
- В модели нет свёрток и реккурентностей, в основе всего лежит механизм внимания и fc-слои
- Лучше, чем RNN, обрабатывают текст большого размера
- RNN плохо параллелится из-за реккурентности, из-за этого сложно обучать

На данный момент Трансформеры (с вариациями) де-факто являются стандартом для решения задач в NLP

3 главные части, благодаря которым Трансформер так хорошо работает:
- Positional Encoding
- Attention
- Self-Attention

Дополнительные видео для ознакомления с темой:

[Видео про Transformer](https://www.youtube.com/watch?v=TQQlZhbC5ps&list=PLTl9hO2Oobd_bzXUpzKMKA3liq2kj6LfE)

[Видео Transformer: Attention layers](https://www.youtube.com/watch?v=FC8PziPmxnQ)

[Видео Transfrormer: Building a Transformer](https://www.youtube.com/watch?v=J4H6A4-dvhE)

## Attention. Повторение

![](images/1.png)

![](images/2.png)

Иллюстрация того, на каких словах акцентируется attention при переводе:

![](images/3.png)

## Self-attention для RNN

Ранее мы видели, что механизм внимания можно применить к Seq2seq моделям. Но на этом применение внимания не ограничивается. Например, можно применить внимание к обычной RNN модели: на шаге 4 кроме hidden-вектора еще используется вектор контекста $c_3$, который получен применением внимания к 3 предыдущим выходам RNN.
Когда добавляется self-attention, RNN меньше "забывает" предыдущую информацию

![](images/4.png)

# Transformer

- Мы использовали механизм внимания в Seq2Seq-моделях для задачи перевода
- Мы также видели выше, что можно применять внимание для обычных RNN (self-attention)
- Можем ли мы вообще убрать RNN?

![](images/5.png)

## Attention layer

Входы энкодера: $x_1, ..., x_m$

Входы декодера: $x^\prime_1, ..., x^\prime_t$

На вход вниманию теперь будут приходить не внутрение состояния экнодера и декодера, а просто их входы.

**Работа внимания**

Обозначения: $W_Q, W_K, W_V$ - это обучаемые матрицы; key и value основаны на входах энкодера $x_1, ..., x_m$, query на входах декодера $x^\prime_1, ..., x^\prime_t$

- query: $q_{:j} = W_Q \cdotp x^\prime_j$ (to match others)
- key: $k_{:i} = W_K · x_i$ (to be matched)
- value: $v_{:i} = W_V · x_i$ (to be weighted averaged)

Из столбцов $k_{:i}$ составляется матрица $K$, а затем коэффициенты взвешенной суммы считаются по формуле: $\alpha_{:j} = Softmax(K^Tq_{:j}) ⊂ ℝ^m$

context vector: $c_j = \alpha_{1j} v_{:1} + ... + \alpha_{mj} v_{:m}$ (всего $c_1, ..., c_t$)

![](images/6.png)

[Откуда взялись Key, Query, Value в Трансформерах](https://telegra.ph/Otkuda-vzyalis-Query-Key-Value-v-transformerah-12-20)

## Attention layer for machine translation



Как мы можем использовать описанный выше attention для задачи перевода? Попробуем порассуждать.

Пусть наша задача - переводить с английского языка на немецкий. Мы хотим сгенерировать 3е слово на немецком. Наш контекстный вектор $c_{:2}$ "знает" обо всём предложении на английском языке и только о 2м слове на немецком. Есть ощущение, что этого недостаточно - мы ничего не знаем про первое слово в результирующем предложении:

![](images/7.png)

Раньше в RNN у нас в декодере был вектор, который обновлялся на каждом time step'е, поэтому мы не теряли информацию о уже переведённой части предложения. Сейчас у нас такого вектора нет, но мы можем вспомнить про self-attention для RNN и использовать этот слой, чтобы не терять информацию

## Self-Attention layer

Мы можем использовать self-attention, как и в примере с RNN. В таком случае коэффициентов $c_1, ..., c_m$ у нас уже будет столько же, как и длина нашей входной последовательности. Здесь, как и в обычном attention, у нас тоже есть обучаемые $W_Q, W_K, W_V$. И key, и value, и query  основаны на входах $x_1, ..., x_m$

## Multi-head attention layer

### Multi-head self-attention

В обычном слое self-attention у нас три матрицы: $W_Q, W_K, W_V$. А сейчас мы хотим, чтобы у нас было $l$ независимых self-attention. Идея в том, что каждая из этих голов учится находить скрытую зависимость между векторами, и если голов много, то они могут обучиться находить разные зависимости (грамматическую зависимость, эмоциональную окраску, смысловую и т.п.). Получается, всего у нас будет $3l$ матриц в таком multi-head self-attention слое. Выходы single-head attention'ов (предположим, они имеют размерность $d\times 1$) мы конкатенируем (получив векторы размера $ld\times 1$)

### Multi-head attention

То же самое, что описано для multi-head self-attention, можно по аналогии проделать для обычного atention

# Building Transformer

- Добавляем к multihead self-attention обычный fc-слой (на рисунке ниже у каждого из `Dense` одинаковые между собой параметры)
- Можем стакать такие слои друг на друга

![](images/8.png)

![](images/9.png)

## Encoder


- На вход энкодеру приходит матрица размера $512\times m$, где $512$ - это размерность эмбеддингов (может быть другим числом), а $m$ - это длина последовательности
- Блоки стакаются друг на друга, между ними есть skip connection'ы
- Веса блоков не зависимы друг от друга.
- Всего 6 блоков, каждый $≃$ multihead self-attention (8 голов) + dense

![](images/10.png)

## Decoder

- Один блок декодера $≃$ multi-head self-attention (8 голов) + multi-head attention (между энкодером и декодером, тоже 8 голов) + dense
- Главное отличие в том, что декодер для внимания использует только те $x^\prime_i$, что уже сгенерированы. Например, для предсказания 3-го слова мы делаем self-attention только с векторами $x^\prime_1, x^\prime_2, x^\prime_3$. В реальной жизни, когда сеть уже натренирована и мы хотим её использовать, следующих слов у нас просто нет. Чтобы сеть чему-то училась, мы симулируем отсутствие следующих векторов, закрывая их маской (поэтому self-attention для декодера называют masked).

![](images/11.png)

![](images/12.png)

### Собираем Трансформер 🤖 🤣

Слева - энкодер, справа - декодер. Выходы энкодера прокидываются в декодер для того, чтобы сделать с ними attention

![](images/13.png)

![](images/14.png)

### Другие особенности модели

- Feed Forward layer
  - Два линейных слоя с нелинейностью (`ReLU`) между ними
  - Дополнительно обрабатывает информацию, полученную при помощи внимания
- Residual connections
  - Обозначена на схеме как `Add` в `Add&Norm`
- Layer Normalization
  - Делает сходимость стабильнее и иногда повышает качество
  ![](images/15.png)
- Positional Encoding
  Мы убрали реккурентность, и теперь у нас Трансформер не знает порядок входных токенов. Например, self-attention вообще инвариантен к перестановкам токенов. Поэтому у нас есть два набора эмбеддингов: для токенов (как обычно) и для позиций - которые мы складываем.
  ![](images/16.png)

### Positional Encoding

В RNN мы подаём слова в сеть друг за другом на каждом time step'е. В Трансформере у нас нет time step'ов, мы подаём все слова, которые есть, вместе разом, и нам надо как-то сохранить информацию о позиции слова в предложении. Для этого к вектору слова прибавляется positional embedding, в котором скрыта информация о позиции. Хочется понять, каким образом этот positional embedding делать.

**Попытка №1**: кодировать int'ом позицию слова

![](images/17.png)

Плюсы:
- Простой и понятный способ

Минусы:
- Для длинной последовательности числа становятся огромные, а мы помним, что мы прибавляем их к векторам. Градиенты взрываются, ничего не обучается 

**Попытка №2**: кодируем первое слово нулём, последнее - единичкой, а остальные пересчитываем линейно 

![](images/18.png)

Плюсы: градиенты не взрываются

Минусы: теперь энкодинг зависит от длины последовательности. Например, в предложениях "Мама мыла" и "Мама мыла раму" для первых двух слов будет разный positional embedding. Это может сбивать с толку сетку, мы не хотим такого

**Попытка №3**: способ, который используется в классическом Трансформере

Как мы уже обсуждали, первый способ с кодированием позиции int'ом был бы идеальным, если бы не взрывались градиенты из-за больших чисел. Однако есть способ закодировать последовательность числами не больше единицы - это перевести номер позиции в двоичную систему. По сути, мы это и делаем, но только в типе данных float:

![](images/20.png)

На рисунке выше видно, что каждый столбец в записи двоичного числа (красный, синий и т.д.) меняет 0 и 1 с какой-то своей периодичностью. Можно представить себе, что каждому столбцу соответствует своя синусоида - например, 0 будет обозначать те моменты времени, когда график синуса выше нуля, а 1 - когда меньше.

Энкодинг позиции в Трансформере, основанный на синусоидальных функциях, можно описать вот такой картинкой (см. ниже). Энкодинг зависит не только от позиции слова в предложении *position*, но ещё и от координаты в эмбеддинге *depth* (если у эмбеддинга размерность 512, то для каждой из 512 будет как бы своя синусоидальная волна, как в картинке с разрядами двоичного числа). В результате получается периодичная структура:

![](images/21.png)

На самом деле используются не синусы, а синусы и косинусы. Объясняется это тем, что в таком случае получается выражать $\mbox{PE}_{pos, j + k}$ линейно выражается через $\mbox{PE}_{pos, j}$ благодаря свойству $sin(j+k) = sin(j)cos(k) + cos(j)sin(k)$

Введём обозначения: $pos$ - это позиция токена, $i$ - это индекс в эмбеддинге, $d_{model}$ - размерность эмбеддинга. Тогда формула для positional embedding будет зависеть от чётности позиции:

  $$\mbox{PE}_{pos, 2i}=\sin(pos/10000^{2i/d_{model}})$$
  $$\mbox{PE}_{pos, 2i+1}=\cos(pos/10000^{2i/d_{model}})$$
  
![](images/19.png)

[Пост про positional encoding](https://kazemnejad.com/blog/transformer_architecture_positional_encoding/)

Примечание: positional encoding можно сделать обучаемым